# Tutorial 0: Unifying a Single Function

In this example, we will unify a function from Kornia, which an awesome computer vision library written in PyTorch.

Specifically, we will unify the function `canny`, which performs Canny edge detection. We will then show how this newly unified `canny` function can be used alongside *any* ML framework.

## Unifying

Firstly, let's import some dependencies

In [ ]:
import ivy
import cv2
import kornia
import numpy as np

Now, let's unify the function!

In [ ]:
canny = ivy.unify(kornia.filters.canny)

And that's it! The `canny` function can now be used with any ML framework. It's as simple as that.

## Testing

So, let's give it a try!

First, let's load an image into `numpy` using `cv2`

In [ ]:
img = cv2.imread('image.png')/255

Let's now convert into the channel-first format with a unary batch dimension, used by Kornia

In [ ]:
img = np.expand_dims(np.transpose(img, (2, 0, 1)), 0)

Now, let's try out our new unified `canny` function with some other ML frameworks!

In [ ]:
# NumPy
canny(img)

# JAX
import jax.numpy as jnp
canny(jnp.array(img))

# TensorFlow
import tensorflow as tf
canny(tf.constant(img))

This new `canny` function is unified because it is implemented in Ivy. In the examples, above no framework has been explicitly set in Ivy's backend, and Ivy automatically determines the backend framework based on the function inputs. However, this doesn't work if the compiled function is purely generative and doesn't consume arrays at all (`ones`, `random_uniform`, `fill` etc.).

In such cases, the backend framework would need to be set explicitly like so:

In [ ]:
ivy.set_backend("jax")

In cases where the target framework is known ahead of time, then this should be set when Ivy is first imported.

## Efficiency

In the examples above, the `canny` function is implemented in Ivy, and all of Ivy's wrapping overhead is included in the call stack which causes runtime overhead. To remove this overhead, the code should be compiled like so:

In [ ]:
canny = ivy.compile(canny)

The canny function will then be stripped down to its constituent functions in the target framework (with Ivy wrapping overhead), and it will also be compiled to machine-code if the target framework supports low-level compiling, for maximal efficiency.

## One Target Framework

A common use case is that there is *one* target framework in mind. Let's assume that you need the `canny` function for a JAX project, and you want it to be efficient. The workflow would then be as follows:

In [ ]:
import ivy
ivy.set_backend("jax")

canny = ivy.compile(ivy.unify(kornia.filters.canny))

`canny` is now compiled to machine-code, specifically for JAX, ready to be integrated into your wider JAX project.